# ESG Deal Screener

This project simulates the workflow of an impact-focused private equity analyst, using 2022 public ESG data to identify and score high-potential sustainable companies.  
While based on listed firms, the approach mirrors how analysts evaluate private or startup deals. Combining data cleaning, simple NLP sector tagging, and composite ESG scoring to surface investment opportunities aligned with CleanTech, AgriTech, and HealthTech themes.  

The goal is to demonstrate practical data science skills in **data preprocessing, NLP, and investment scoring**, while exploring how ESG analytics can support sustainable investing decisions.


### 1. Inspecting the dataset

We start by loading the CSV file and checking the structure of the data to understand which columns will be useful for scoring, NLP tagging, and visualisation.


In [19]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)


In [20]:
file_path = "../data/esg_data_with_descriptions.csv"
df = pd.read_csv(file_path)


df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722 entries, 0 to 721
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ticker                722 non-null    object
 1   name                  722 non-null    object
 2   currency              722 non-null    object
 3   exchange              722 non-null    object
 4   industry              709 non-null    object
 5   logo                  702 non-null    object
 6   weburl                706 non-null    object
 7   environment_grade     722 non-null    object
 8   environment_level     722 non-null    object
 9   social_grade          722 non-null    object
 10  social_level          722 non-null    object
 11  governance_grade      722 non-null    object
 12  governance_level      722 non-null    object
 13  environment_score     722 non-null    int64 
 14  social_score          722 non-null    int64 
 15  governance_score      722 non-null    in

,ticker,name,currency,exchange,industry,logo,weburl,environment_grade,environment_level,social_grade,social_level,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik,description
0,dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/ef50b4a2b263c84...,https://thewaltdisneycompany.com/,A,High,BB,Medium,BB,Medium,510,316,321,1147,19-04-2022,BBB,High,1744489,Walt Disney Co operates in the Media sector an...
1,gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Automobiles,https://static.finnhub.io/logo/9253db78-80c9-1...,https://www.gm.com/,A,High,BB,Medium,B,Medium,510,303,255,1068,17-04-2022,BBB,High,1467858,General Motors Co operates in the Automobiles ...
2,gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Trading Companies and Distributors,https://static.finnhub.io/logo/f153dcda-80eb-1...,https://www.grainger.com/,B,Medium,BB,Medium,B,Medium,255,385,240,880,19-04-2022,BB,Medium,277135,WW Grainger Inc operates in the Trading Compan...
3,mhk,Mohawk Industries Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,https://static.finnhub.io/logo/26868a62-80ec-1...,https://mohawkind.com/,A,High,B,Medium,BB,Medium,570,298,303,1171,18-04-2022,BBB,High,851968,Mohawk Industries Inc operates in the Consumer...
4,lyv,Live Nation Entertainment Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/1cd144d2-80ec-1...,https://www.livenationentertainment.com/,BBB,High,BB,Medium,B,Medium,492,310,250,1052,18-04-2022,BBB,High,1335258,Live Nation Entertainment Inc operates in the ...


### 2. Handling missing values and duplicates

Before any analysis, we ensure data consistency by checking for missing or duplicate rows.  
Only 13 missing values were found in the *industry* column which we fill with 'Unknown' to preserve all records.


In [21]:
# Checking for missing values and duplicates
print(df.isna().sum())
print(f"\nDuplicate rows: {df.duplicated().sum()}")

df['industry'] = df['industry'].fillna('Unknown')


df['industry'].isna().sum(), df['industry'].value_counts().head()


ticker                   0
name                     0
currency                 0
exchange                 0
industry                13
                        ..
last_processing_date     0
total_grade              0
total_level              0
cik                      0
description              0
Length: 22, dtype: int64

Duplicate rows: 0


(np.int64(0),
 industry
 Technology            63
 Biotechnology         56
 Health Care           51
 Real Estate           36
 Financial Services    33
 Name: count, dtype: int64)

### 3. Converting dates to datetime format

Even though the last_processing_date column appears to contain dates, pandas initially reads it as text.  
We convert it to a datetime object to enable sorting, filtering, and the creation of a recency score later.


In [22]:
# Convert 'last_processing_date' to datetime
df['last_processing_date'] = pd.to_datetime(
    df['last_processing_date'],
    errors='coerce',
    dayfirst=True
)


# Verify conversion
df['last_processing_date'].describe

print(df['last_processing_date'].min())
print(df['last_processing_date'].max())


2022-04-16 00:00:00
2022-11-15 00:00:00


In [23]:
df['industry'] = (
    df['industry']
    .astype(str)
    .str.strip()
    .str.title()     
)


In [24]:
df.head()
df.describe(include='all')


,ticker,name,currency,exchange,industry,logo,weburl,environment_grade,environment_level,social_grade,social_level,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik,description
count,722,722,722,722,722,702,706,722,722,722,722,722,722,722.000000,722.000000,722.000000,722.000000,722,722,722,7.220000e+02,722
unique,722,722,5,2,47,702,706,5,3,6,4,4,3,NaN,NaN,NaN,NaN,NaN,4,2,NaN,722
top,dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Technology,https://static.finnhub.io/logo/ef50b4a2b263c84...,https://thewaltdisneycompany.com/,A,High,BB,Medium,BB,Medium,NaN,NaN,NaN,NaN,NaN,BBB,High,NaN,Walt Disney Co operates in the Media sector an...
freq,1,1,704,377,63,1,1,321,366,441,703,434,716,NaN,NaN,NaN,NaN,NaN,368,451,NaN,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404.806094,292.182825,278.761773,975.750693,2022-05-13 12:09:58.337950208,NaN,NaN,9.897925e+05,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240.000000,243.000000,235.000000,763.000000,2022-04-17 00:00:00,NaN,NaN,7.231572e+05,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,483.000000,302.000000,300.000000,1046.000000,2022-04-18 00:00:00,NaN,NaN,1.046189e+06,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,518.750000,322.750000,310.000000,1144.000000,2022-06-10 00:00:00,NaN,NaN,1.470094e+06,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,719.000000,667.000000,475.000000,1536.000000,2022-11-15 00:00:00,NaN,NaN,1.914023e+06,NaN


In [25]:
df['description'].head(10)


0    Walt Disney Co operates in the Media sector an...
1    General Motors Co operates in the Automobiles ...
2    WW Grainger Inc operates in the Trading Compan...
3    Mohawk Industries Inc operates in the Consumer...
4    Live Nation Entertainment Inc operates in the ...
5    Las Vegas Sands Corp operates in the Hotels Re...
6    Clorox Co operates in the Consumer products se...
7    ATA Creativity Global operates in the Diversif...
8    American Airlines Group Inc operates in the Ai...
9    Atlantic American Corp operates in the Insuran...
Name: description, dtype: object

### Normalising for NLP 

In [26]:
df['description'].isna().sum()


np.int64(0)

In [27]:
df['description'] = (
    df['description']
    .astype(str)
    .str.lower()
    .str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
    .str.strip()
)


### Add Descriptions for NLP

This phase ensures that every company has a meaningful, ESG-flavoured textual description suitable for natural language processing (NLP).  

1. Check if a description column exists and fill missing values  
2. Clean the text (lowercasing, punctuation removal)  
3. Enrich it with simulated ESG tone where needed


In [28]:
df['description'] = df.apply(
    lambda row: (
        f"{row['name']} operates in the {row['industry']} sector and is known for its commitment to sustainable innovation and ethical governance."
        if pd.isna(row['description']) or row['description'].strip() == ''
        else row['description']
    ),
    axis=1
)


In [29]:
df['description'].head(5)


0    walt disney co operates in the media sector an...
1    general motors co operates in the automobiles ...
2    ww grainger inc operates in the trading compan...
3    mohawk industries inc operates in the consumer...
4    live nation entertainment inc operates in the ...
Name: description, dtype: object

### 5. Saving the enriched dataset

In [30]:
df.to_csv("../data/esg_data_enriched.csv", index=False)
